In [2]:
import pandas as pd
import re

In [3]:
import os
os.chdir('/Users/qas/Desktop/CSA/ES-CSA/data/raw')

cdrs = pd.read_csv('CDRS.csv')
purchases = pd.read_csv('Purchases.csv')
tickets = pd.read_csv('Tickets.csv')
userprofile = pd.read_csv('UserProfile.csv')

In [4]:
# CDRS

cdrs['Datetime Charged'] = pd.to_datetime(cdrs['Datetime Charged'], format='%Y-%m-%d %H:%M:%S') #convert to datetime

def classify_resource(resource_str): #function to standardize the resource type
    if 'SMS' in resource_str:
        return 'SMS'
    elif 'seconds voice call' in resource_str:
        return 'Voice Call'
    elif 'MB data' in resource_str:
        return 'Data'
    else:
        return 'Unknown'

cdrs['Resource Type'] = cdrs['Resources Consumed'].apply(classify_resource) #standarize the resource type

def extract_resource_value(resource_str): #function to extract the resource value
    match = re.search(r'\d+', resource_str)
    return int(match.group()) if match else None

cdrs['Resource Value'] = cdrs['Resources Consumed'].apply(extract_resource_value) #extract the resource value

cdrs.drop(columns=['Resources Consumed'], inplace=True) #drop the original column
cdrs.tail()

,Amount Charged,Datetime Charged,MSISDN,Resource Type,Resource Value
4995,2,2024-11-02 06:31:04,9230582711549,SMS,13
4996,0,2023-01-22 07:40:10,9230515578106,Data,68
4997,6,2024-04-10 00:02:43,9230585087525,Data,54
4998,0,2023-10-04 01:42:28,9230150956349,Data,4
4999,0,2023-02-13 07:43:37,9230771370719,Voice Call,272


In [5]:
# Purchases

purchases['Datetime'] = pd.to_datetime(purchases['Datetime'], format='%Y-%m-%d %H:%M:%S') #convert to datetime

purchases.head()


,Offer Name,Offer ID,Data Browsing Allowance,SMS Allowance,Voice On-Net Allowance,Voice Off-Net Allowance,Data Social Allowance,Datetime,Amount,MSISDN
0,Offer 27,O031,3035,979,81,181,1236,2024-04-13 11:05:30,604,9230715340104
1,Offer 42,O020,4819,676,218,40,1226,2023-09-05 11:29:10,356,9230636066934
2,Offer 30,O017,7907,565,80,195,2691,2023-03-11 13:05:07,248,9230570122595
3,Offer 12,O001,4495,496,104,216,1418,2023-05-17 00:56:48,737,9230996629579
4,Offer 34,O047,3851,237,157,188,2975,2024-11-14 13:06:23,766,9230403240894


In [6]:
# Tickets

tickets['Log Time'] = pd.to_datetime(tickets['Log Time'], format='%Y-%m-%d %H:%M:%S') #convert to datetime
tickets['Resolution Time'] = pd.to_datetime(tickets['Resolution Time'], format='%Y-%m-%d %H:%M:%S') #convert to datetime

tickets.head()

,Ticket ID,Log Time,Resolution Time,Category,Description,Resolutions,MSISDN
0,T54509,2024-07-19 10:30:02,2024-07-19 16:30:02,Network Issue,Issue reported under Network Issue category.,Resolved with detailed explanation for Network...,9230323952441
1,T66986,2023-11-29 05:45:02,2023-12-02 01:45:02,Network Issue,Issue reported under Network Issue category.,Resolved with detailed explanation for Network...,9230367979227
2,T73025,2023-02-28 21:52:29,2023-03-03 16:52:29,General Inquiry,Issue reported under General Inquiry category.,Resolved with detailed explanation for General...,9230277146142
3,T15861,2023-02-09 08:34:25,2023-02-10 13:34:25,Network Issue,Issue reported under Network Issue category.,Resolved with detailed explanation for Network...,9230488909528
4,T90855,2023-04-07 06:07:08,2023-04-09 22:07:08,Network Issue,Issue reported under Network Issue category.,Resolved with detailed explanation for Network...,9230224556190


In [7]:
# UserProfile

userprofile.drop(columns=['Age'], inplace=True) #drop the age column for redundancy
userprofile.drop(columns=['Gender'], inplace=True) #drop the gender column for redundancy

userprofile.head()

,Name,City,User Type,MSISDN
0,User 1,Lahore,Prepaid,9230610000463
1,User 2,Quetta,Postpaid,9230347659110
2,User 3,Karachi,Prepaid,9230141002657
3,User 4,Karachi,Postpaid,9230162731400
4,User 5,Islamabad,Prepaid,9230108284824


In [8]:
combined_data = userprofile.merge(purchases, on='MSISDN', how='left') #merge userprofile and purchases
combined_data = combined_data.merge(cdrs, on='MSISDN', how='left') #merge with cdrs
combined_data = combined_data.merge(tickets, on='MSISDN', how='left') #merge with tickets


combined_data.to_csv('/Users/qas/Desktop/CSA/ES-CSA/data/combined/combined_data.csv', index=False) #saving the combined data

In [9]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501606 entries, 0 to 501605
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Name                     501606 non-null  object        
 1   City                     501606 non-null  object        
 2   User Type                501606 non-null  object        
 3   MSISDN                   501606 non-null  int64         
 4   Offer Name               501606 non-null  object        
 5   Offer ID                 501606 non-null  object        
 6   Data Browsing Allowance  501606 non-null  int64         
 7   SMS Allowance            501606 non-null  int64         
 8   Voice On-Net Allowance   501606 non-null  int64         
 9   Voice Off-Net Allowance  501606 non-null  int64         
 10  Data Social Allowance    501606 non-null  int64         
 11  Datetime                 501606 non-null  datetime64[ns]
 12  Amount          